# Sentiment Analysis of Cryptocurrencies using Tweets
Himani Anil Deshpande

Downloading Tweepy 

In [ ]:
!pip3 install tweepy --upgrade

  Using cached tweepy-4.7.0-py2.py3-none-any.whl (76 kB)
  Using cached oauthlib-3.2.0-py3-none-any.whl (151 kB)
  Using cached requests-2.27.1-py2.py3-none-any.whl (63 kB)
  Using cached certifi-2021.10.8-py2.py3-none-any.whl (149 kB)
     |████████████████████████████████| 138 kB 5.6 MB/s eta 0:00:01
  Using cached idna-3.3-py3-none-any.whl (61 kB)


Importing Libraries

In [ ]:
import tweepy
import pandas as pd
import json
from pprint import pprint
import requests
import re

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/himanideshpande/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords


Creating Client for Twitter API v2 which return request.Response as output and keeping the rate limit on


In [ ]:
client = tweepy.Client(bearer_token='AAAAAAAAAAAAAAAAAAAAACo7agEAAAAAnmtWpmzPvoLe%2B6NzCWXe7l8AVak%3DFp8XZBjy0MX6X8i85R9dBzoNURWxJ80y9FAnwpLgeA8HHcDkxX', wait_on_rate_limit=True,return_type= requests.Response)


Setting query parameters 

In [ ]:
tweet_fields = ["geo", "created_at", "id", "text", "author_id", "public_metrics", "source"]
expansions = ["geo.place_id"]
place_fields = ["country", "full_name", "contained_within"]


Building a query which is searched

In [ ]:
query = '((#bitcoin) OR (#bitcoins) OR bitcoins OR bitcoin  OR (#BTC) OR (#XBT) OR (#satoshi) OR XBT OR ($XBT) OR (#BTC) OR ($BTC)) place_country:US has:geo has:mentions -is:retweet lang:en'
         

Working with the 3 timeline of Before, During and After the event


In [ ]:
Timeline = {"Before":("2021-01-28", "2021-01-26"), "During": ("2021-01-31", "2021-01-29"),"After":("2021-02-03", "2021-02-01") }
# (EndTime, StartTime) Before , During ,After


Function to fetch Archived Tweets

In [ ]:

def get_tweets(endTime , startTime,token= None ):
    return client.search_all_tweets(query = query,  end_time=endTime +"T00:00:00Z", start_time=startTime +"T00:00:00Z",
                         expansions=expansions, max_results=500,  place_fields=place_fields, 
                                  sort_order= ['relevancy'], next_token= token,
                           tweet_fields=tweet_fields)

Saving the Raw data in a Json file

In [ ]:
def save_raw_data(file_name, data):
    json_object = json.dumps(data, indent = 4)

    with open(file_name, "w") as f:
        f.write(json_object)

Converting the JSON output in a Data Frame

In [ ]:
def convert_json_to_df(tweets_json_data):
    
    data_tweet_df = pd.json_normalize(tweets_json_data['data'])#, record_path =['students'])
    geo_df = pd.json_normalize(tweets_json_data['includes']['places'])#, record_path =['students']))
    geo_df.rename(columns= {'id':'geo.place_id'}, inplace = True)
    combined_data_df = pd.merge(data_tweet_df,geo_df, on = 'geo.place_id')
    return combined_data_df

## Fetching Data from Twitter for the 3 timelines

In [ ]:
records_cnt = 0
token = None
full_combined_df = None
for k,v in Timeline.items():
    endTime, startTime = v
#   Getting Tweets from the 3 Timelines
    tweets_json = get_tweets( endTime=endTime , startTime= startTime, token = token).json()
#     Saving the Raw JSON data
    save_raw_data("raw-data-page-"+k+".json", tweets_json)
#     Getting a count of records to verify Later with the shape of full dataframe
    records_cnt += tweets_json['meta']['result_count']
#     token= tweets_json['meta']['next_token']
    
#     Converting JSON data into Dataframe and Saving them in CSV format
    combined_data_df = convert_json_to_df(tweets_json)
    combined_data_df['Timeline'] = k
    combined_data_df.to_csv("raw_dataFrame_page_"+k+".csv")
#     Combining two Pages of Tweets's Data frames into one
    full_combined_df = pd.concat([full_combined_df,combined_data_df])

Rate limit exceeded. Sleeping for 901 seconds.


In [ ]:
full_combined_df.shape

(819, 15)

In [ ]:
records_cnt

819

In [ ]:
full_combined_df

,created_at,text,source,id,author_id,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,geo.place_id,country,full_name,Timeline,geo.coordinates.type,geo.coordinates.coordinates
0,2021-01-26T19:01:49.000Z,Excited to be doing the Keynote virtually at T...,Twitter for iPhone,1354142516828889095,29468585,7,5,39,0,be930565864872db,United States,"Sardinera, Puerto Rico",Before,NaN,NaN
1,2021-01-26T20:03:11.000Z,The amount of activity on #Tezos right now com...,Twitter for Android,1354157962424709123,17816729,14,0,44,0,01a9a39529b27f36,United States,"Manhattan, NY",Before,NaN,NaN
2,2021-01-27T17:10:54.000Z,@dannydevan Only if there was some sort of cur...,Twitter for iPhone,1354476991848787971,592375643,0,0,2,0,01a9a39529b27f36,United States,"Manhattan, NY",Before,NaN,NaN
3,2021-01-27T23:50:18.000Z,@redditinvestors As soon as #silver starting t...,Twitter for iPhone,1354577502732230659,20944170,0,0,3,0,01a9a39529b27f36,United States,"Manhattan, NY",Before,NaN,NaN
4,2021-01-26T12:32:58.000Z,@souljaboy Bitcoin. Just stack your #BTC and ...,Twitter for iPhone,1354044660218474497,940577363695472640,0,0,0,0,01a9a39529b27f36,United States,"Manhattan, NY",Before,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,2021-02-01T19:38:56.000Z,@haileykb .0125 bitcoin,Twitter for Android,1356326183730868227,476359600,0,1,0,0,59b0a84a9f7451d5,United States,"North Branch, MN",After,NaN,NaN
183,2021-02-02T23:00:52.000Z,@BarrettSallee Would your opinion change if yo...,Twitter for iPhone,1356739390127636484,80618949,0,0,0,0,8173485c72e78ca5,United States,"Atlanta, GA",After,NaN,NaN
184,2021-02-01T14:29:00.000Z,"@DocumentingBTC If it’s basically zero, it’s a...",Twitter for iPhone,1356248186575331329,891464868,0,0,0,0,87f5cf8444a63c94,United States,"Wamego, KS",After,NaN,NaN
185,2021-02-01T14:58:30.000Z,@HatsumiNonaka Bitcoin,Twitter for iPhone,1356255610073993218,1238878102673076224,0,0,0,0,be925761a3187f65,United States,"Greenacres, FL",After,NaN,NaN


Saving the Full Data we extracted from Twitter in a CSV file

In [ ]:
full_combined_df.to_csv("All_data_without_scores.csv")

Creating an extract of Columns that i will be working on


In [ ]:
full_combined_df.columns


Index(['created_at', 'text', 'source', 'id', 'author_id',
       'public_metrics.retweet_count', 'public_metrics.reply_count',
       'public_metrics.like_count', 'public_metrics.quote_count',
       'geo.place_id', 'country', 'full_name', 'Timeline',
       'geo.coordinates.type', 'geo.coordinates.coordinates'],
      dtype='object')

In [ ]:
full_combined_df = full_combined_df[['created_at', 'text', 'source',
       'public_metrics.retweet_count', 'public_metrics.reply_count',
       'public_metrics.like_count', 'public_metrics.quote_count',
        'country', 'full_name', 'Timeline']]

In [ ]:
full_combined_df

,created_at,text,source,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,country,full_name,Timeline
0,2021-01-26T19:01:49.000Z,Excited to be doing the Keynote virtually at T...,Twitter for iPhone,7,5,39,0,United States,"Sardinera, Puerto Rico",Before
1,2021-01-26T20:03:11.000Z,The amount of activity on #Tezos right now com...,Twitter for Android,14,0,44,0,United States,"Manhattan, NY",Before
2,2021-01-27T17:10:54.000Z,@dannydevan Only if there was some sort of cur...,Twitter for iPhone,0,0,2,0,United States,"Manhattan, NY",Before
3,2021-01-27T23:50:18.000Z,@redditinvestors As soon as #silver starting t...,Twitter for iPhone,0,0,3,0,United States,"Manhattan, NY",Before
4,2021-01-26T12:32:58.000Z,@souljaboy Bitcoin. Just stack your #BTC and ...,Twitter for iPhone,0,0,0,0,United States,"Manhattan, NY",Before
...,...,...,...,...,...,...,...,...,...,...
182,2021-02-01T19:38:56.000Z,@haileykb .0125 bitcoin,Twitter for Android,0,1,0,0,United States,"North Branch, MN",After
183,2021-02-02T23:00:52.000Z,@BarrettSallee Would your opinion change if yo...,Twitter for iPhone,0,0,0,0,United States,"Atlanta, GA",After
184,2021-02-01T14:29:00.000Z,"@DocumentingBTC If it’s basically zero, it’s a...",Twitter for iPhone,0,0,0,0,United States,"Wamego, KS",After
185,2021-02-01T14:58:30.000Z,@HatsumiNonaka Bitcoin,Twitter for iPhone,0,0,0,0,United States,"Greenacres, FL",After


## Data Cleaning

Removing URL's and Usernames 

In [ ]:


def clean_usernames_links(tweet):
    
    tweet = re.sub('http[^\s]+','',tweet)
    tweet = re.sub('@[^\s]+','',tweet)
    return tweet



In [ ]:
full_combined_df['cleaned_tweet_text'] = full_combined_df['text'].apply(clean_usernames_links)


/var/folders/58/p264y9qj0ts1vgjcf8qzxb0r0000gn/T/ipykernel_29758/2137147158.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_combined_df['cleaned_tweet_text'] = full_combined_df['text'].apply(clean_usernames_links)


In [ ]:
full_combined_df

,created_at,text,source,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,country,full_name,Timeline,cleaned_tweet_text
0,2021-01-26T19:01:49.000Z,Excited to be doing the Keynote virtually at T...,Twitter for iPhone,7,5,39,0,United States,"Sardinera, Puerto Rico",Before,Excited to be doing the Keynote virtually at T...
1,2021-01-26T20:03:11.000Z,The amount of activity on #Tezos right now com...,Twitter for Android,14,0,44,0,United States,"Manhattan, NY",Before,The amount of activity on #Tezos right now com...
2,2021-01-27T17:10:54.000Z,@dannydevan Only if there was some sort of cur...,Twitter for iPhone,0,0,2,0,United States,"Manhattan, NY",Before,Only if there was some sort of currency that ...
3,2021-01-27T23:50:18.000Z,@redditinvestors As soon as #silver starting t...,Twitter for iPhone,0,0,3,0,United States,"Manhattan, NY",Before,As soon as #silver starting trending the suit...
4,2021-01-26T12:32:58.000Z,@souljaboy Bitcoin. Just stack your #BTC and ...,Twitter for iPhone,0,0,0,0,United States,"Manhattan, NY",Before,Bitcoin. Just stack your #BTC and come back ...
...,...,...,...,...,...,...,...,...,...,...,...
182,2021-02-01T19:38:56.000Z,@haileykb .0125 bitcoin,Twitter for Android,0,1,0,0,United States,"North Branch, MN",After,.0125 bitcoin
183,2021-02-02T23:00:52.000Z,@BarrettSallee Would your opinion change if yo...,Twitter for iPhone,0,0,0,0,United States,"Atlanta, GA",After,Would your opinion change if you found out th...
184,2021-02-01T14:29:00.000Z,"@DocumentingBTC If it’s basically zero, it’s a...",Twitter for iPhone,0,0,0,0,United States,"Wamego, KS",After,"If it’s basically zero, it’s actually negativ..."
185,2021-02-01T14:58:30.000Z,@HatsumiNonaka Bitcoin,Twitter for iPhone,0,0,0,0,United States,"Greenacres, FL",After,Bitcoin


## VADER Sentiment Analysis

In [ ]:

nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/himanideshpande/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
full_combined_df['clean_scores'] = full_combined_df['cleaned_tweet_text'].apply(lambda x: sid.polarity_scores(x))
# full_combined_df['unclean_scores'] = full_combined_df['text'].apply(lambda x: sid.polarity_scores(x))

/var/folders/58/p264y9qj0ts1vgjcf8qzxb0r0000gn/T/ipykernel_29758/92348387.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_combined_df['clean_scores'] = full_combined_df['cleaned_tweet_text'].apply(lambda x: sid.polarity_scores(x))


In [ ]:
full_combined_df

,created_at,text,source,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,country,full_name,Timeline,cleaned_tweet_text,clean_scores
0,2021-01-26T19:01:49.000Z,Excited to be doing the Keynote virtually at T...,Twitter for iPhone,7,5,39,0,United States,"Sardinera, Puerto Rico",Before,Excited to be doing the Keynote virtually at T...,"{'neg': 0.0, 'neu': 0.921, 'pos': 0.079, 'comp..."
1,2021-01-26T20:03:11.000Z,The amount of activity on #Tezos right now com...,Twitter for Android,14,0,44,0,United States,"Manhattan, NY",Before,The amount of activity on #Tezos right now com...,"{'neg': 0.0, 'neu': 0.903, 'pos': 0.097, 'comp..."
2,2021-01-27T17:10:54.000Z,@dannydevan Only if there was some sort of cur...,Twitter for iPhone,0,0,2,0,United States,"Manhattan, NY",Before,Only if there was some sort of currency that ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
3,2021-01-27T23:50:18.000Z,@redditinvestors As soon as #silver starting t...,Twitter for iPhone,0,0,3,0,United States,"Manhattan, NY",Before,As soon as #silver starting trending the suit...,"{'neg': 0.204, 'neu': 0.74, 'pos': 0.056, 'com..."
4,2021-01-26T12:32:58.000Z,@souljaboy Bitcoin. Just stack your #BTC and ...,Twitter for iPhone,0,0,0,0,United States,"Manhattan, NY",Before,Bitcoin. Just stack your #BTC and come back ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
...,...,...,...,...,...,...,...,...,...,...,...,...
182,2021-02-01T19:38:56.000Z,@haileykb .0125 bitcoin,Twitter for Android,0,1,0,0,United States,"North Branch, MN",After,.0125 bitcoin,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
183,2021-02-02T23:00:52.000Z,@BarrettSallee Would your opinion change if yo...,Twitter for iPhone,0,0,0,0,United States,"Atlanta, GA",After,Would your opinion change if you found out th...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
184,2021-02-01T14:29:00.000Z,"@DocumentingBTC If it’s basically zero, it’s a...",Twitter for iPhone,0,0,0,0,United States,"Wamego, KS",After,"If it’s basically zero, it’s actually negativ...","{'neg': 0.346, 'neu': 0.654, 'pos': 0.0, 'comp..."
185,2021-02-01T14:58:30.000Z,@HatsumiNonaka Bitcoin,Twitter for iPhone,0,0,0,0,United States,"Greenacres, FL",After,Bitcoin,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."


In [ ]:
def separate_columns(data, column_str, column):
    data[column_str+'_neg'] = data[column].apply(lambda x: x['neg'])
    data[column_str+'_neu'] = data[column].apply(lambda x: x['neu'])
    data[column_str+'_pos'] = data[column].apply(lambda x: x['pos'])
    data[column_str+'_compound'] = data[column].apply(lambda x: x['compound'])
    return data


In [ ]:
full_combined_df = separate_columns(full_combined_df, 'clean', 'clean_scores')
# full_combined_df = separate_columns(full_combined_df, 'unclean', 'unclean_scores')

/var/folders/58/p264y9qj0ts1vgjcf8qzxb0r0000gn/T/ipykernel_29758/305574733.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_str+'_neg'] = data[column].apply(lambda x: x['neg'])
/var/folders/58/p264y9qj0ts1vgjcf8qzxb0r0000gn/T/ipykernel_29758/305574733.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_str+'_neu'] = data[column].apply(lambda x: x['neu'])


In [ ]:
full_combined_df

,created_at,text,source,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,country,full_name,Timeline,cleaned_tweet_text,clean_scores,clean_neg,clean_neu,clean_pos,clean_compound
0,2021-01-26T19:01:49.000Z,Excited to be doing the Keynote virtually at T...,Twitter for iPhone,7,5,39,0,United States,"Sardinera, Puerto Rico",Before,Excited to be doing the Keynote virtually at T...,"{'neg': 0.0, 'neu': 0.921, 'pos': 0.079, 'comp...",0.000,0.921,0.079,0.3400
1,2021-01-26T20:03:11.000Z,The amount of activity on #Tezos right now com...,Twitter for Android,14,0,44,0,United States,"Manhattan, NY",Before,The amount of activity on #Tezos right now com...,"{'neg': 0.0, 'neu': 0.903, 'pos': 0.097, 'comp...",0.000,0.903,0.097,0.4201
2,2021-01-27T17:10:54.000Z,@dannydevan Only if there was some sort of cur...,Twitter for iPhone,0,0,2,0,United States,"Manhattan, NY",Before,Only if there was some sort of currency that ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000
3,2021-01-27T23:50:18.000Z,@redditinvestors As soon as #silver starting t...,Twitter for iPhone,0,0,3,0,United States,"Manhattan, NY",Before,As soon as #silver starting trending the suit...,"{'neg': 0.204, 'neu': 0.74, 'pos': 0.056, 'com...",0.204,0.740,0.056,-0.6808
4,2021-01-26T12:32:58.000Z,@souljaboy Bitcoin. Just stack your #BTC and ...,Twitter for iPhone,0,0,0,0,United States,"Manhattan, NY",Before,Bitcoin. Just stack your #BTC and come back ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,2021-02-01T19:38:56.000Z,@haileykb .0125 bitcoin,Twitter for Android,0,1,0,0,United States,"North Branch, MN",After,.0125 bitcoin,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000
183,2021-02-02T23:00:52.000Z,@BarrettSallee Would your opinion change if yo...,Twitter for iPhone,0,0,0,0,United States,"Atlanta, GA",After,Would your opinion change if you found out th...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000
184,2021-02-01T14:29:00.000Z,"@DocumentingBTC If it’s basically zero, it’s a...",Twitter for iPhone,0,0,0,0,United States,"Wamego, KS",After,"If it’s basically zero, it’s actually negativ...","{'neg': 0.346, 'neu': 0.654, 'pos': 0.0, 'comp...",0.346,0.654,0.000,-0.5719
185,2021-02-01T14:58:30.000Z,@HatsumiNonaka Bitcoin,Twitter for iPhone,0,0,0,0,United States,"Greenacres, FL",After,Bitcoin,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000


 Each tweet can be classified as having an overall negative, neutral, or positive sentiment polarity based on its compound sentiment score. We consider any text with a compound VADER sentiment score below −0.05 as having an overall negative polarity, above 0.05, as having an overall positive polarity. Other scores have an overall neutral polarity. 


Link https://www.mdpi.com/2504-2289/4/4/33/htm
<!-- AMA Style -->
Pano T, Kashef R. A Complete VADER-Based Sentiment Analysis of Bitcoin (BTC) Tweets during the Era of COVID-19. Big Data and Cognitive Computing. 2020; 4(4):33. https://doi.org/10.3390/bdcc4040033



Function to get the Final Classification of the Sentiment using VADER polarity score

In [ ]:
def final_score_clasification(data, column_name, final_column_name):
    data[final_column_name] = data[column_name].apply(lambda x: 'negative' if x < -0.05 else ('positive' if x > 0.05  else 'neutral'))
    return data

In [ ]:
full_combined_df = final_score_clasification(full_combined_df, 'clean_compound', 'clean_final_class')
# full_combined_df = final_score_clasification(full_combined_df, 'unclean_compound', 'unclean_final_class')

I performed VADER on both clean and raw tweet content and found that the polarity scores doesnt change much but there is a variation in the individual scores.

In [ ]:
full_combined_df

,created_at,text,source,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,country,full_name,Timeline,cleaned_tweet_text,clean_scores,clean_neg,clean_neu,clean_pos,clean_compound,clean_final_class
0,2021-01-26T19:01:49.000Z,Excited to be doing the Keynote virtually at T...,Twitter for iPhone,7,5,39,0,United States,"Sardinera, Puerto Rico",Before,Excited to be doing the Keynote virtually at T...,"{'neg': 0.0, 'neu': 0.921, 'pos': 0.079, 'comp...",0.000,0.921,0.079,0.3400,positive
1,2021-01-26T20:03:11.000Z,The amount of activity on #Tezos right now com...,Twitter for Android,14,0,44,0,United States,"Manhattan, NY",Before,The amount of activity on #Tezos right now com...,"{'neg': 0.0, 'neu': 0.903, 'pos': 0.097, 'comp...",0.000,0.903,0.097,0.4201,positive
2,2021-01-27T17:10:54.000Z,@dannydevan Only if there was some sort of cur...,Twitter for iPhone,0,0,2,0,United States,"Manhattan, NY",Before,Only if there was some sort of currency that ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,neutral
3,2021-01-27T23:50:18.000Z,@redditinvestors As soon as #silver starting t...,Twitter for iPhone,0,0,3,0,United States,"Manhattan, NY",Before,As soon as #silver starting trending the suit...,"{'neg': 0.204, 'neu': 0.74, 'pos': 0.056, 'com...",0.204,0.740,0.056,-0.6808,negative
4,2021-01-26T12:32:58.000Z,@souljaboy Bitcoin. Just stack your #BTC and ...,Twitter for iPhone,0,0,0,0,United States,"Manhattan, NY",Before,Bitcoin. Just stack your #BTC and come back ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,2021-02-01T19:38:56.000Z,@haileykb .0125 bitcoin,Twitter for Android,0,1,0,0,United States,"North Branch, MN",After,.0125 bitcoin,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,neutral
183,2021-02-02T23:00:52.000Z,@BarrettSallee Would your opinion change if yo...,Twitter for iPhone,0,0,0,0,United States,"Atlanta, GA",After,Would your opinion change if you found out th...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,neutral
184,2021-02-01T14:29:00.000Z,"@DocumentingBTC If it’s basically zero, it’s a...",Twitter for iPhone,0,0,0,0,United States,"Wamego, KS",After,"If it’s basically zero, it’s actually negativ...","{'neg': 0.346, 'neu': 0.654, 'pos': 0.0, 'comp...",0.346,0.654,0.000,-0.5719,negative
185,2021-02-01T14:58:30.000Z,@HatsumiNonaka Bitcoin,Twitter for iPhone,0,0,0,0,United States,"Greenacres, FL",After,Bitcoin,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,neutral


In [ ]:
full_combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 819 entries, 0 to 186
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   created_at                    819 non-null    object 
 1   text                          819 non-null    object 
 2   source                        819 non-null    object 
 3   public_metrics.retweet_count  819 non-null    int64  
 4   public_metrics.reply_count    819 non-null    int64  
 5   public_metrics.like_count     819 non-null    int64  
 6   public_metrics.quote_count    819 non-null    int64  
 7   country                       819 non-null    object 
 8   full_name                     819 non-null    object 
 9   Timeline                      819 non-null    object 
 10  cleaned_tweet_text            819 non-null    object 
 11  clean_scores                  819 non-null    object 
 12  clean_neg                     819 non-null    float64
 13  clean

 Getting Date from the timestamp

In [ ]:
full_combined_df['created_at'] = full_combined_df['created_at'].astype('datetime64[ns]')
full_combined_df['created_at_date'] = full_combined_df['created_at'].dt.date

In [ ]:
full_combined_df

,created_at,text,source,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,country,full_name,Timeline,cleaned_tweet_text,clean_scores,clean_neg,clean_neu,clean_pos,clean_compound,clean_final_class,created_at_date
0,2021-01-26 19:01:49,Excited to be doing the Keynote virtually at T...,Twitter for iPhone,7,5,39,0,United States,"Sardinera, Puerto Rico",Before,Excited to be doing the Keynote virtually at T...,"{'neg': 0.0, 'neu': 0.921, 'pos': 0.079, 'comp...",0.000,0.921,0.079,0.3400,positive,2021-01-26
1,2021-01-26 20:03:11,The amount of activity on #Tezos right now com...,Twitter for Android,14,0,44,0,United States,"Manhattan, NY",Before,The amount of activity on #Tezos right now com...,"{'neg': 0.0, 'neu': 0.903, 'pos': 0.097, 'comp...",0.000,0.903,0.097,0.4201,positive,2021-01-26
2,2021-01-27 17:10:54,@dannydevan Only if there was some sort of cur...,Twitter for iPhone,0,0,2,0,United States,"Manhattan, NY",Before,Only if there was some sort of currency that ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,neutral,2021-01-27
3,2021-01-27 23:50:18,@redditinvestors As soon as #silver starting t...,Twitter for iPhone,0,0,3,0,United States,"Manhattan, NY",Before,As soon as #silver starting trending the suit...,"{'neg': 0.204, 'neu': 0.74, 'pos': 0.056, 'com...",0.204,0.740,0.056,-0.6808,negative,2021-01-27
4,2021-01-26 12:32:58,@souljaboy Bitcoin. Just stack your #BTC and ...,Twitter for iPhone,0,0,0,0,United States,"Manhattan, NY",Before,Bitcoin. Just stack your #BTC and come back ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,neutral,2021-01-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,2021-02-01 19:38:56,@haileykb .0125 bitcoin,Twitter for Android,0,1,0,0,United States,"North Branch, MN",After,.0125 bitcoin,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,neutral,2021-02-01
183,2021-02-02 23:00:52,@BarrettSallee Would your opinion change if yo...,Twitter for iPhone,0,0,0,0,United States,"Atlanta, GA",After,Would your opinion change if you found out th...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,neutral,2021-02-02
184,2021-02-01 14:29:00,"@DocumentingBTC If it’s basically zero, it’s a...",Twitter for iPhone,0,0,0,0,United States,"Wamego, KS",After,"If it’s basically zero, it’s actually negativ...","{'neg': 0.346, 'neu': 0.654, 'pos': 0.0, 'comp...",0.346,0.654,0.000,-0.5719,negative,2021-02-01
185,2021-02-01 14:58:30,@HatsumiNonaka Bitcoin,Twitter for iPhone,0,0,0,0,United States,"Greenacres, FL",After,Bitcoin,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,neutral,2021-02-01


Removing stopwords from The clean tweet text so that I can use it for creating a wordcloud and ca use for EmoLex

In [ ]:
stopwords = stopwords.words('english')

In [ ]:
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [ ]:
full_combined_df['tweet_no_stopwords']= full_combined_df['cleaned_tweet_text'].apply(lambda x: ' '.join([word for word in x.lower().split() if word not in (stopwords)]))



In [ ]:
full_combined_df

,created_at,text,source,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,country,full_name,Timeline,cleaned_tweet_text,clean_scores,clean_neg,clean_neu,clean_pos,clean_compound,clean_final_class,created_at_date,tweet_no_stopwords
0,2021-01-26 19:01:49,Excited to be doing the Keynote virtually at T...,Twitter for iPhone,7,5,39,0,United States,"Sardinera, Puerto Rico",Before,Excited to be doing the Keynote virtually at T...,"{'neg': 0.0, 'neu': 0.921, 'pos': 0.079, 'comp...",0.000,0.921,0.079,0.3400,positive,2021-01-26,excited keynote virtually north american bitco...
1,2021-01-26 20:03:11,The amount of activity on #Tezos right now com...,Twitter for Android,14,0,44,0,United States,"Manhattan, NY",Before,The amount of activity on #Tezos right now com...,"{'neg': 0.0, 'neu': 0.903, 'pos': 0.097, 'comp...",0.000,0.903,0.097,0.4201,positive,2021-01-26,amount activity #tezos right compared starting...
2,2021-01-27 17:10:54,@dannydevan Only if there was some sort of cur...,Twitter for iPhone,0,0,2,0,United States,"Manhattan, NY",Before,Only if there was some sort of currency that ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,neutral,2021-01-27,sort currency financial institutions can’t man...
3,2021-01-27 23:50:18,@redditinvestors As soon as #silver starting t...,Twitter for iPhone,0,0,3,0,United States,"Manhattan, NY",Before,As soon as #silver starting trending the suit...,"{'neg': 0.204, 'neu': 0.74, 'pos': 0.056, 'com...",0.204,0.740,0.056,-0.6808,negative,2021-01-27,soon #silver starting trending suits got invol...
4,2021-01-26 12:32:58,@souljaboy Bitcoin. Just stack your #BTC and ...,Twitter for iPhone,0,0,0,0,United States,"Manhattan, NY",Before,Bitcoin. Just stack your #BTC and come back ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,neutral,2021-01-26,bitcoin. stack #btc come back years.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,2021-02-01 19:38:56,@haileykb .0125 bitcoin,Twitter for Android,0,1,0,0,United States,"North Branch, MN",After,.0125 bitcoin,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,neutral,2021-02-01,.0125 bitcoin
183,2021-02-02 23:00:52,@BarrettSallee Would your opinion change if yo...,Twitter for iPhone,0,0,0,0,United States,"Atlanta, GA",After,Would your opinion change if you found out th...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,neutral,2021-02-02,would opinion change found compensated bitcoin?
184,2021-02-01 14:29:00,"@DocumentingBTC If it’s basically zero, it’s a...",Twitter for iPhone,0,0,0,0,United States,"Wamego, KS",After,"If it’s basically zero, it’s actually negativ...","{'neg': 0.346, 'neu': 0.654, 'pos': 0.0, 'comp...",0.346,0.654,0.000,-0.5719,negative,2021-02-01,"it’s basically zero, it’s actually negative. $btc"
185,2021-02-01 14:58:30,@HatsumiNonaka Bitcoin,Twitter for iPhone,0,0,0,0,United States,"Greenacres, FL",After,Bitcoin,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,neutral,2021-02-01,bitcoin


Saving the output of VADER sentiment analysis in a CSV File

In [ ]:
full_combined_df.to_csv("All_data_with_no_stopwords_polarity_score_VADER.csv")

# NRC Sentiment Analysis (EmoLex)

In [ ]:
pip install NRCLex

Note: you may need to restart the kernel to use updated packages.


In [ ]:
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to
[nltk_data]     /Users/himanideshpande/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/himanideshpande/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/himanideshpande/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/himanideshpande/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to
[nltk_data]     /Users/himanideshpande/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/himanideshpande/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


In [ ]:
from nrclex import NRCLex



In [ ]:
full_combined_df['cleaned_tweet_text_NRC'] = full_combined_df['cleaned_tweet_text'].apply(lambda x : NRCLex(x).affect_frequencies)



In [ ]:
full_combined_df['cleaned_tweet_text_NRC']

0      {'fear': 0.0, 'anger': 0.0, 'anticip': 0.0, 't...
1      {'fear': 0.0, 'anger': 0.0, 'anticip': 0.0, 't...
2      {'fear': 0.0, 'anger': 0.0, 'anticip': 0.0, 't...
3      {'fear': 0.15384615384615385, 'anger': 0.15384...
4      {'fear': 0.0, 'anger': 0.0, 'anticip': 0.0, 't...
                             ...                        
182    {'fear': 0.0, 'anger': 0.0, 'anticip': 0.0, 't...
183    {'fear': 0.25, 'anger': 0.0, 'anticip': 0.0, '...
184    {'fear': 0.0, 'anger': 0.0, 'anticip': 0.0, 't...
185    {'fear': 0.0, 'anger': 0.0, 'anticip': 0.0, 't...
186    {'fear': 0.0, 'anger': 0.0, 'anticip': 0.0, 't...
Name: cleaned_tweet_text_NRC, Length: 819, dtype: object

In [ ]:
full_combined_df.shape

(819, 20)

In [ ]:
emotions = ['fear', 'anger', 'anticip', 'trust', 'surprise', 'positive', 'negative', 'sadness', 'disgust', 'joy']

In [ ]:
def separate_columns(data, column_str, column, emotion):
    data[column_str+'_'+emotion] = data[column].apply(lambda x: x[emotion])
    
    return data


In [ ]:
for emo in emotions:
    full_combined_df = separate_columns(full_combined_df, 'clean', 'cleaned_tweet_text_NRC', emo)

In [ ]:
full_combined_df.shape

(819, 30)

Saving Output of EmoLex

In [ ]:
full_combined_df.to_csv("NRC_Lexical_analysis.csv")

References:

https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-all

https://docs.tweepy.org/en/stable/client.html#tweets

https://developer.twitter.com/en/docs/twitter-api/tweets/counts/integrate/build-a-query

https://stackoverflow.com/questions/50830214/remove-usernames-from-twitter-data-using-python


https://pypi.org/project/NRCLex/

https://github.com/cjhutto/vaderSentiment